# Dokumentation des Vorverarbeitungs- und Trainingsprozesses

## 1. Einführung

Dieses Dokument beschreibt den **Datenvorverarbeitungsprozess** und das **Training des Modells** zur Hautkrebserkennung basierend auf Bildern. Es werden die folgenden Schritte behandelt:

1. **Laden und Bereinigen des Datensatzes**
2. **Anwendung von Transformationen und Datenaugmentierung**
3. **Normalisierung der Metadaten**
4. **Aufteilung in Trainings-, Validierungs- und Testdaten**
5. **Training des Modells mit PyTorch**
6. **Umgang mit Early Stopping und Speichern des besten Modells**

---

## 2. Projekteinrichtung

- **Datensatz:** `data/ISIC-images`
- **Bereinigte Metadatendatei:** `data/metadata_clean_20250226_222654.csv`
- **Bildgröße:** `(224, 224)`
- **Speicherort des trainierten Modells:** `models/best_model.pth`

---

## 3. Datenvorverarbeitung

### 3.1 Laden des Datensatzes

Die bereinigte CSV-Datei wird geladen, und vorhandene Bilder werden gefiltert.

```python
# Metadaten laden
import pandas as pd
import os

CSV_PATH = "data/metadata_clean_20250226_222654.csv"
DATASET_PATH = "data/ISIC-images"

df = pd.read_csv(CSV_PATH)
df["img_path"] = df["isic_id"].apply(lambda x: os.path.join(DATASET_PATH, f"{x}.jpg"))
df = df[df["img_path"].apply(os.path.exists)].reset_index(drop=True)
```

### 3.2 Datenaugmentierung und Transformationen

Es wird **Albumentations** verwendet, um Transformationen auf die Bilder anzuwenden:

- **Rotationen und Spiegelungen**: Simuliert verschiedene Ausrichtungen.
- **Helligkeit und Kontrast**: Verbessert die Variabilität der Hauttöne.
- **GaussianBlur und CoarseDropout**: Simuliert unscharfe oder verdeckte Bildbereiche.
- **ElasticTransform**: Verformt das Bild leicht, um die Robustheit des Modells zu verbessern.

```python
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=20, p=0.5),
    A.GaussianBlur(blur_limit=(3, 5), p=0.2),
    A.CoarseDropout(num_holes=1, max_holes=2, max_height=20, max_width=20, p=0.3),
    A.ElasticTransform(alpha=1, sigma=50, p=0.2),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
```

### 3.3 Normalisierung der Metadaten

Die Variablen `age_approx` und `sex` werden mit `StandardScaler` normalisiert.

```python
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
X_metadata = scaler.fit_transform(df[["age_approx", "sex"]])
joblib.dump(scaler, "models/scaler.pkl")
```

---

## 4. Aufteilung der Daten in Trainings-, Validierungs- und Testdaten

Die Daten werden in Trainings- (70%), Validierungs- (15%) und Testdaten (15%) aufgeteilt.

```python
from sklearn.model_selection import train_test_split
import numpy as np

X_train_img, X_temp_img, X_train_meta, X_temp_meta, y_train, y_temp = train_test_split(
    X_images, X_metadata, y, test_size=0.3, random_state=42, stratify=y)

X_val_img, X_test_img, X_val_meta, X_test_meta, y_val, y_test = train_test_split(
    X_temp_img, X_temp_meta, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
```

Die vorverarbeiteten Daten werden gespeichert:

```python
np.save("data/X_train_img.npy", X_train_img)
np.save("data/X_train_meta.npy", X_train_meta)
np.save("data/y_train.npy", y_train)
```

---

## 5. Training des Modells mit PyTorch

### 5.1 Trainingskonfiguration

```python
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from model import SkinCancerModel

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-4
PATIENCE = 5
BEST_MODEL_PATH = "models/best_model.pth"
```

### 5.2 Definition des Modells

```python
model = SkinCancerModel(num_classes=2).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
```

### 5.3 Training mit Early Stopping

```python
best_val_loss = float("inf")
epochs_no_improve = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    
    for images, metadata, labels in train_loader:
        images, metadata, labels = images.to(DEVICE), metadata.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images, metadata)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    val_loss = 0
    model.eval()
    with torch.no_grad():
        for images, metadata, labels in val_loader:
            outputs = model(images, metadata)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print("🛑 Early stopping aktiviert")
            break
```

---

## 6. Fazit

- **Die Bilder und Metadaten wurden korrekt vorverarbeitet.**
- **Der Datensatz wurde in Trainings-, Validierungs- und Testdaten aufgeteilt**, um Overfitting zu vermeiden.
- **Das Modell wurde mit Early Stopping trainiert**, und der beste Checkpoint wurde gespeichert.
- **Die Pipeline ist modular und wiederverwendbar** für zukünftige Modelle.

**Das Modell ist bereit für Tests und weitere Optimierungen.**

---

**Hinweis**: Diese Dokumentation ist als lebendes Dokument gedacht. Wenn sich das Modell weiterentwickelt, werden Aktualisierungen vorgenommen, um neue Funktionen, Verbesserungen und Änderungen in den Best Practices widerzuspiegeln.